# RNN 有記憶的神經網路
要記得前面發生的事，將前一次的輸出當成這一次的輸入

我們終於要介紹三大神經網路的最後一個, 也就是 RNN。RNN 有不少的變型, 例如 LSTM 和 GRU 等等, 不過我們都通稱叫 RNN。RNN 是一種「有記憶」的神經網路, 非常適合時間序列啦, 或是不定長度的輸入資料。

有三個閥門 像神經元

我們來看看怎麼樣用 RNN 做電影評論的「情意分析」, 也就是知道一則評論究竟是「正評」還是「負評」。

## 01 初始準備

In [1]:
%env KERAS_BACKEND=tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

## 02 讀入 IMDB 電影數據庫
今天我們要評入 IMDB 電影數據庫影評的部份。

In [3]:
from keras.datasets import imdb

Using TensorFlow backend.


In [4]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=10000)

17465344/17464789 [==============================] - 82s 5us/step


### 輸入資料部份
我們來看一下輸入部份長什麼樣子?

要注意這裡我們限制只選「最常用」1 萬字, 也就是超過這範圍的就當不存在。這是文字分析常會做的事。

In [6]:
len(x_train)

25000

In [7]:
len(x_test)

25000

注意這其實是一個 list 而不是 array, 原因是每筆資料 (每段影評) 長度自然是不一樣的! 我們檢查一下前 10 筆的長度就可以知道。

In [8]:
x_train[999]

[1,
 2,
 47,
 1678,
 72,
 19,
 61,
 205,
 2,
 5,
 2,
 146,
 6,
 655,
 2212,
 1021,
 336,
 7,
 107,
 146,
 179,
 2,
 11,
 61,
 965,
 5,
 1624,
 113,
 131,
 13,
 28,
 1460,
 39,
 61,
 501,
 15,
 13,
 361,
 2,
 8,
 2,
 5,
 1341,
 39,
 63,
 13,
 244,
 1364,
 9026,
 54,
 134,
 2,
 5,
 68,
 2166,
 1497,
 68,
 456,
 19,
 72,
 36,
 339,
 72,
 8,
 4392,
 61,
 113,
 5,
 61,
 649,
 19,
 61,
 223,
 10,
 10,
 4,
 123,
 1025,
 111,
 409,
 2,
 83,
 4,
 501,
 7,
 4,
 6176,
 5,
 2672,
 687,
 15,
 242,
 2164,
 68,
 6644,
 111,
 7,
 178,
 2712,
 88,
 45,
 99,
 629,
 8,
 140,
 145,
 5,
 81,
 17,
 2676,
 3088,
 560,
 4,
 3808,
 5,
 921,
 5607,
 7,
 4,
 882,
 11,
 4,
 2267,
 5,
 955,
 479,
 7,
 263,
 1548,
 4,
 123,
 1015,
 6,
 176,
 7,
 1109,
 18,
 33,
 222,
 397,
 14,
 1773,
 645,
 14,
 7847,
 9,
 1350,
 5,
 878,
 21,
 290,
 12,
 38,
 76,
 9029,
 7,
 6644,
 2615,
 5,
 701,
 2615,
 186,
 8,
 2754,
 4,
 5029,
 1341,
 400,
 45,
 5015,
 15,
 4,
 6176,
 43,
 31,
 251,
 645,
 8,
 1258,
 56,
 42,
 845,
 18,
 253

In [9]:
len(x_train[999])

434

In [10]:
len(x_train[99])

171

## 輸出的部分

輸出方面應該很容易想像, 我們來看看前 10 筆。結果自然就是 0 (負評) 或 1 (正評)。

In [15]:
y_train[:10]

array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0])

In [11]:
y_train[999]

1

## 資料處理
### 送入神經網路的輸入處理
雖然 RNN 是可以處理不同長度的輸入, 在寫程式時我們還是要

* 設輸入文字長度的上限
* 把每段文字都弄成一樣長, 太短的後面補上 0

In [12]:
from keras.preprocessing import sequence

In [13]:
x_train = sequence.pad_sequences(x_train, maxlen=150)
x_test = sequence.pad_sequences(x_test, maxlen=150)

In [14]:
x_train.shape

(25000, 150)

## 03 設計我們的情意分析 RNN¶
這裡我們選用 LSTM, 基本上用哪種 RNN 寫法都是差不多的!


### 決定神經網路架構
先將 10000 維的文字壓到 N 維
然後用 K 個 LSTM 神經元做隱藏層
最後一個 output, 直接用 sigmoid 送出

### 建構我們的神經網路
文字我們用 1-hot 表示是很標準的方式, 不過要注意的是, 因為我們指定要 1 萬個字, 所以每個字是用 1 萬維的向量表示! 這一來很浪費記憶空間, 二來字和字間基本上是沒有關係的。我們可以用某種「合理」的方式, 把字壓到比較小的維度, 這些向量又代表某些意思 (比如說兩個字代表的向量角度小表相關程度大) 等等。

這聽來很複雜的事叫 "word embedding", 而事實上 Keras 會幫我們做。我們只需告訴 Keras 原來最大的數字是多少 (10000), 還有我們打算壓到幾維 (N)。

* 10000維 -> 壓到3維(自訂)
* RNN:10
* Dense(1)

In [17]:
N = 3 # 文字要壓到 N 維
K = 10 # LSTM 有 K 個神經元

LSTM, GRU 都是做RNN的

In [16]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding

### Step01. 建置RNN神經網路

In [18]:
model = Sequential()

In [19]:
model.add(Embedding(10000, N))

LSTM 層, 我們做 K 個 LSTM Cells。

In [20]:
model.add(LSTM(K))

sigmoid 將數值壓到0-1之間

In [21]:
model.add(Dense(1, activation='sigmoid'))

### 組裝

這次我們用 binary_crossentropy 做我們的 loss function, 另外用一個很潮的 Adam 學習法。

Adam會自動調整learning rate

In [22]:
model.compile(loss='binary_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 3)           30000     
_________________________________________________________________
lstm_1 (LSTM)                (None, 10)                560       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 30,571
Trainable params: 30,571
Non-trainable params: 0
_________________________________________________________________


* RNN: 

    4x(13x10+10(bias))

有三個閥門 加本身

In [26]:
4*(13*10+10)

560

### Step02. 訓練

我們用的 embedding 中, 會被 batch_size 影響輸入。輸入的 shape 會是

    (batch_size, 每筆上限)

也就是 (32,100) 輸出是 (32,100,128), 其中 128 是我們決定要壓成幾維的向量。

In [27]:
model.fit(x_train, y_train,
         batch_size=32,
         epochs=7)

/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
25000/25000 [==============================] - 48s 2ms/step - loss: 0.4881 - accuracy: 0.7520
Epoch 2/10
25000/25000 [==============================] - 46s 2ms/step - loss: 0.2711 - accuracy: 0.8948
Epoch 3/10
25000/25000 [==============================] - 46s 2ms/step - loss: 0.2136 - accuracy: 0.9195
Epoch 4/10
25000/25000 [==============================] - 46s 2ms/step - loss: 0.1779 - accuracy: 0.9378
Epoch 5/10
25000/25000 [==============================] - 45s 2ms/step - loss: 0.1560 - accuracy: 0.9460
Epoch 6/10
25000/25000 [==============================] - 42s 2ms/step - loss: 0.1321 - accuracy: 0.9555
Epoch 7/10
25000/25000 [==============================] - 41s 2ms/step - loss: 0.1183 - accuracy: 0.9602
Epoch 8/10
25000/25000 [==============================] - 41s 2ms/step - loss: 0.1031 - accuracy: 0.9673
Epoch 9/10
25000/25000 [==============================] - 44s 2ms/step - loss: 0.0907 - accuracy: 0.9713
Epoch 10/10
25000/25000 [==============================

### Step03. 檢視結果
#### 分數
我們照例來看看測試資料的分數。

In [28]:
score = model.evaluate(x_test, y_test)

25000/25000 [==============================] - 10s 402us/step


In [29]:
print(f'測試資料的 loss = {score[0]}')
print(f'測試資正確率 = {score[1]}')

測試資料的 loss = 0.5919722938346863
測試資正確率 = 0.8369600176811218


In [30]:
score

[0.5919722938346863, 0.8369600176811218]

#### 儲存結果
這裡有 8 成我們可以正確分辨, 看來還不差, 照例我們把結果存檔。

In [32]:
model_json = model.to_json()
open('imdb_model_arch.json','w').write(model_json)
model.save_weights('imdb_model_weights.h5')

### Q&A

In [33]:
x_test[333]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          1, 4700, 2755, 5696,    2,   39,  658,   18,  701, 3383, 1007,
         63,   26,  684,  532, 2472,   23,    4, 1320,   59,  461,    2,
         19,    2, 1224, 5939, 1138, 2319,  323,    5, 6062, 4946,   21,
       4595, 1002,    4,  552,    7,   20,   15,  100,   64,   28,   77,
          2,   11,    4, 3141,  290,    6,  168,   18,   94,  701, 2063,
        114,    5,   18,  402,  354,   34,  705,  381,   10,   10, 1332,
       3473,  127,   24,  193,  245,   99,   76,   39,  360,    7,  509,
         63,  287,   35,  221, 5228,    7,  113], d

## Talk to transformer

## GPT-2
## Open AI